In [ ]:
from langchain_community.document_loaders.pdf import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_chroma import Chroma

In [ ]:
paths = [
    "files/apostila.pdf",
    "files/LLM.pdf",
    ]

pages = []
for path in paths:
    loader = PyPDFLoader(path)
    pages.extend(loader.load())

recur_split = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=100,
    separators=["\n\n", "\n", ".", " ", ""]
)

documents = recur_split.split_documents(pages)



In [ ]:
for i, doc in enumerate(documents):
    doc.metadata['source'] = doc.metadata['source'].replace('arquivos/', '')
    doc.metadata['doc_id'] = i


path = 'arquivos/chat_retrieval_db'

embeddings_model = OpenAIEmbeddings()
vectordb = Chroma.from_documents(
    documents=documents,
    embedding=embeddings_model,
    persist_directory=path
)

In [ ]:
from langchain_openai.chat_models import ChatOpenAI
from langchain.chains.retrieval_qa.base import RetrievalQA

chat = ChatOpenAI(model="gpt-4o-mini")

chat_chain = RetrievalQA.from_chain_type(
    llm=chat,
    retriever=vectordb.as_retriever(search_type='mmr'),
)

In [ ]:
question = "O que é Hugging Face e como faço para acessá-lo?"
result = chat_chain.run(question)
print(result)

In [ ]:
from langchain.prompts import PromptTemplate

chain_prompt = PromptTemplate.from_template(
"""Utilize o contexto fornecido para responder a pergunta ao final. 
Se você não sabe a resposta, apenas diga que não sabe e não invente uma resposta.
Utilize três frases no máximo, mantenha a resposta concisa.
Fale em gírias nordestinas e de gangster, pois você é um gangster nordestino estiloso.

Contexto: {context}

Pergunta: {question}

Resposta:
"""
)

chat_chain = RetrievalQA.from_chain_type(
    llm=chat,
    retriever=vectordb.as_retriever(search_type='mmr'), # mmr = MMR também procura relevância, mas evita repetição.
    # chain_type_kwargs={"prompt": chain_prompt},
    chain_type='refine', # 'refine' permite que o modelo refine a resposta com base em múltiplos documentos. Mas o chain_type_kwargs tem que ser removido.
    return_source_documents=True # Retorna os documentos de origem que foram usados para gerar a resposta.
)

question = "O que é Hugging Face e como faço para acessá-lo?"
result = chat_chain.invoke({"query": question})
print(result["result"])
print(result["source_documents"])

ValidationError: 1 validation error for RefineDocumentsChain
prompt
  extra fields not permitted (type=value_error.extra)

In [ ]:
# DEBUGGING
# Para ver os logs de depuração, você pode usar o seguinte código:
from langchain.globals import set_debug

set_debug(True)

pergunta = 'O que é Hugging Face e como faço para acessá-lo?'
resposta = chat_chain.invoke({'query': pergunta})

set_debug(False)